In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from string import ascii_letters
import re
import nltk
import string
import time

In [2]:
import TextCleaning
import importlib

In [3]:
# reading in raw data

df = pd.read_csv('../../data/original/raw_abstracts.csv',engine='python')
print(df.shape)

(550088, 16)


In [4]:
# this will be handled by Lara's script
df = TextCleaning.remove_nulls(df, "ABSTRACT")
df = TextCleaning.remove_duplicates(df)

3 nulls in  ABSTRACT . These rows removed.
11 duplicate abstracts removed
0 project ID duplicates - not removed


In [5]:
# lowercase all abstracts, strip leading and trailing whitespace,
# and save in a working abstract column that will be updated as text is cleaned

df["working_abstract"] = [abstract.lower().strip() for abstract in df["ABSTRACT"]] 
df = TextCleaning.drop_empties(df)

wa = 'working_abstract'

Int64Index([], dtype='int64')
dropped 0


In [6]:
df['Start Char']=df['working_abstract'].apply(lambda x: x[0])

In [7]:
df.head()

,PROJECT_ID,ABSTRACT,FY,FIRST_CHAR,LAST_CHAR,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_TITLE,PROJECT_TERMS,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,ORGANIZATION_NAME,CFDA_CODE,FY_TOTAL_COST,working_abstract,Start Char
0,89996,"This is a project to explore Game-based, Metap...",2008,"This is a project to explore Game-based, Metap...",.,NSF,NSF,NaN,0814512,RUI: CYGAMES: CYBER-ENABLED TEACHING AND LEARN...,Achievement; analog; base; Cognitive Science; ...,"REESE, DEBBIE D","CARTER, BEVERLY; WOOD, CHARLES; HITT, BEN",WHEELING JESUIT UNIVERSITY,47.076,1999467.0,"this is a project to explore game-based, metap...",t
1,89997,Institution: Franklin Institute Science Museum...,2008,Institution: Franklin Institute Science Museum...,.,NSF,NSF,NaN,0741659,ARIEL - AUGMENTED REALITY FOR INTERPRETIVE AND...,Active Learning; Child; Computer software; des...,"SNYDER, STEVEN","ELINICH, KAREN; YOON, SUSAN",FRANKLIN INSTITUTE,47.076,1799699.0,institution: franklin institute science museum...,i
2,89998,Through programs (including small group conver...,2008,Through programs (including small group conver...,.,NSF,NSF,NaN,0813522,BRIGHTER FUTURES: PUBLIC DELIBERATION ABOUT TH...,Address; Age; Birth; Brain; Caregivers; Child;...,"FINK, LAURIE KLEINBAUM","CADIGAN, KAREN; ELLENBOGEN, KIRSTEN",SCIENCE MUSEUM OF MINNESOTA,47.076,1505858.0,through programs (including small group conver...,t
3,89999,In partnership with the American Chemical Soci...,2008,In partnership with the American Chemical Soci...,.,NSF,NSF,NaN,0838627,FOSTERING US-INTERNATIONAL COLLABORATIVE PARTN...,Advanced Development; American; Chemicals; Che...,"JOST, JOHN W","MILLER, BRADLEY; BOWMAN, KATHERINE",INTERNATIONAL UNION OF PURE AND APPLIED CHEMISTRY,47.049,51000.0,in partnership with the american chemical soci...,i
4,90000,Amphibian populations around the world are exp...,2008,Amphibian populations around the world are exp...,.,NSF,NSF,NaN,0815315,COLLABORATIVE RESEARCH: EVOLUTION OF AMPHIBIAN...,Amphibia; Central America; Communicable Diseas...,"ZAMUDIO, KELLY R",NaN,CORNELL UNIVERSITY ITHACA,47.074,370996.0,amphibian populations around the world are exp...,a


In [5]:
## what do we want to do here???

#importlib.reload(TextCleaning)
df = TextCleaning.remove_short_abstracts(df,limit=150)

1270 short abstracts removed


### Cleaning Strategy:
1. Remove abstracts with all non-alphanumeric characters.
2. Remove non-alphanumeric characters from the start and end of abstracts
3. Remove other non-readable abstracts. (REMOVAL ABSTRACTS FOUND BY INSPECTION)
4. Remove "junk" starting strings and ending strings
5. Remove "junk" strings in the middle 
6. Remove title and organization name from abstracts


In [8]:
# strip non-alphnum characters from the beginning and end of each abstract

df["working_abstract"] = [TextCleaning.strip_nonalnum(abstract) for abstract in df["working_abstract"]]
df = TextCleaning.drop_empties(df)

Int64Index([92077, 445788, 477487], dtype='int64')
dropped 3


In [9]:
# remove abstracts that are not readable: THIS NEEDS TO BE UPDATED BY HAND FOR EVERY NEW DATASET
# For example: index = 490684: ¢ £/¥ ƒ § ¤ ƒ “ ƒ « ...

df["Start Char"] = df['working_abstract'].apply(lambda x: x[0])
ix = df[df['Start Char'] == 'ƒ'].index
df.drop(index = ix, inplace = True)

In [10]:
dict(df["Start Char"].value_counts())

{'d': 219964,
 't': 136215,
 'p': 43877,
 'a': 29678,
 'i': 15636,
 's': 12932,
 'c': 12292,
 'w': 10662,
 'o': 9418,
 'm': 8505,
 'b': 7607,
 'n': 6869,
 'r': 5585,
 'e': 5124,
 'f': 4112,
 'h': 3793,
 'u': 3269,
 '1': 3232,
 'g': 3159,
 'l': 2761,
 '0': 1320,
 'v': 1265,
 'k': 564,
 '7': 512,
 'j': 437,
 'q': 338,
 '6': 154,
 'y': 153,
 '2': 143,
 'x': 109,
 'z': 98,
 '3': 98,
 '5': 58,
 '4': 48,
 '9': 46,
 '8': 33,
 'α': 1,
 'β': 1,
 'γ': 1,
 'δ': 1}

In [11]:
# "junk" phrases at start to remove

start_phrases=['abstract', 'summary', 'proposal', 'description', 'narrative', 
               'technical abstract',
               'non technical abstract', 
               'non- technical abstract',
               'non-technical abstract',                      
               'nontechnical abstract',
               'technical summary', 
               'nontechnical summary',
               'non-technical summary',
               'non-technical description',
               'description (provided by the applicant)',
               'description (provided by investigator)',  
               'description (provided by applicant)',
               'project summary/abstract',
               'proposal abstract',
               'research abstract',
               'project summary',
               'research summary',
               'project description'
               'see instructions):',
               'for center application (provided by the investigator):',
               'objective(s)',      
               'exceed the space provided',
               'provided by applicant',
               'provided by candidate']
                
 # [hrd #######]


In [12]:
#Remove found start phrases
for phrase in start_phrases:
    df[wa]=df[wa].apply(TextCleaning.remove_phrase,args=[phrase,'Start']) 

# strip non-alphanum characters from the beginning and end of each abstract

df["working_abstract"] = [TextCleaning.strip_nonalnum(abstract) for abstract in df["working_abstract"]]
df = TextCleaning.drop_empties(df)    
    

Int64Index([86769, 188021, 262748, 437435, 438719, 453984], dtype='int64')
dropped 6


In [13]:
#Repeated start phrase removal in case the order of project summary/abstract varies

for phrase in start_phrases:
    df[wa]=df[wa].apply(TextCleaning.remove_phrase,args=[phrase,'Start'])  


In [14]:
#starting_exact_phrases to remove

#'This subproject represents an estimate of the percentage of the CTSA funding that isbeing utilized for a broad area of research (AIDS research, pediatric research, orclinical trials).  The Total Cost listed is only an estimate of the amount of CTSAinfrastructure going towards this area of research, not direct funding provided bythe NCRR grant to the subproject or subproject staff.'
#'This subproject is one of many research subprojects utilizing theresources provided by a Center grant funded by NIH/NCRR. The subproject andinvestigator (PI) may have received primary funding from another NIH source,and thus could be represented in other CRISP entries. The institution listed isfor the Center, which is not necessarily the institution for the investigator.'

df[wa]=df[wa].apply(lambda x: x.replace('this subproject represents an estimate of the percentage of the ctsa funding that isbeing utilized for a broad area of research (aids research, pediatric research, orclinical trials).  the total cost listed is only an estimate of the amount of ctsainfrastructure going towards this area of research, not direct funding provided bythe ncrr grant to the subproject or subproject staff.',
                                       ''))

expression=re.compile('this subproject is one of many research subprojects.*not necessarily the institution for the investigator.')
df[wa]=df[wa].apply(lambda x: re.sub(expression,'',x))

expression=re.compile('this subproject is one of many research subprojects.*to the subproject or subproject staff.')
df[wa]=df[wa].apply(lambda x: re.sub(expression,'',x))
  

In [15]:
# strip non-alphnum characters from the beginning and end of each abstract

df["working_abstract"] = [TextCleaning.strip_nonalnum(abstract) for abstract in df["working_abstract"]]
df = TextCleaning.drop_empties(df)
    
# update Start Char column in df
df['Start Char']=df[wa].apply(lambda x: x[0])

Int64Index([34372, 405975], dtype='int64')
dropped 2


In [16]:
end_phrases = ['(end of abstract',
               'end of abstract', 
               '(abstract end',  
               '(end of abstract',
               '(end 0f abstract',
               '(end of absract',
               '(abstract below',
               '(end of reviewers\' comment',
               '(end abstract',
               'performance site ========================================section end',
               'key personnel ========================================section end',
               '[summary truncated at 7800 characters', 
               'this award reflects nsf\'s statutory mission and has been deemed worthy of support through evaluation using the foundation\'s intellectual merit and broader impacts review criteria',
               'project description page 6', 'page 1 of 1', 'project summary/abstract page 6',
               'project description page 7', 'project summary/abstract page 7', 'pag 1 o 1', 
               'page 2 number pages consecutively at the bottom throughout form page 2']


In [17]:
# end phrase removal

for phrase in end_phrases:
    df[wa]=df[wa].apply(TextCleaning.remove_phrase,args=[phrase,'End'])  

# strip non-alphanum characters from the beginning and end of each abstract

df["working_abstract"] = [TextCleaning.strip_nonalnum(abstract) for abstract in df["working_abstract"]]
df = TextCleaning.drop_empties(df)
    
# update Last Char column in df
df['LAST_CHAR']=df[wa].apply(lambda x: x[-1])

Int64Index([], dtype='int64')
dropped 0


In [18]:
# "junk" removal within text body - not necessarily at the start or end

# 1. 'Enter the text here that' ending with 'lines of text.'
expression=re.compile('enter the text here that.*lines of text')
df[wa]=df[wa].apply(lambda x: re.sub(expression,'',x))

expression=re.compile('phs .*?continuation format page')
df[wa]=df[wa].apply(lambda x: re.sub(expression,'',x))

expression=re.compile('omb no .*?continuation format page')
df[wa]=df[wa].apply(lambda x: re.sub(expression,'',x))

df[wa]=df[wa].replace('project summary/abstract','')


In [19]:
"""If it starts with 'one page and must contain',
This is an NIH thing and there aren't that many of them, but come from 3 different cfda
it will start with "one page and must contain a summary of the proposed activity suitable for dissemination to thepublic. 
It should be a self-contained description of the project and should contain a statement of objectives and methods to be employed.
It should be informative to other persons working in the same or related fields and insofar as possible understandable to a technically liter-ate lay reader. 
This Abstract must not include any proprietary/confidential information.* 
Please click the add attachment button to complete this entry." plus some attachments, which includes tracking number, twice:
following the second trackign number, there is a grant number followed by the actual content" 

At the end of these files, they all end in 'Project Narrative File'(last instance) followed by more attachments, all of which can be discarded
"""
expression1=re.compile('one page and must.*?tracking number.*?(tracking number)')
expression2=re.compile('project narrative file.*')

def fix_abstract(abstract):
    if abstract.startswith('one page and must contain'):
        abstract=re.sub(expression1,'',abstract)
        return re.sub(expression2,'',abstract)
    else:
        return abstract

df[wa]=df[wa].apply(fix_abstract)


In [20]:
# removal of phrase at end or beginning?

expression=re.compile('project summary/abstract page.*')

def remove_contact_pd(x):
    
    # removes clause at end that tends to occur: eg Project Summary/Abstract Page 222Contact PD/PI: Sampson, HughNarrative (
    
    if x.startswith('Contact PD/PI'):
        return re.sub(expression,'',x)
    else:
        return x

df[wa]=df[wa].apply(remove_contact_pd) 

In [21]:
# strip non-alphanum characters from the beginning and end of each abstract

df["working_abstract"] = [TextCleaning.strip_nonalnum(abstract) for abstract in df["working_abstract"]]
df = TextCleaning.drop_empties(df)

Int64Index([376319], dtype='int64')
dropped 1


In [ ]:
def remove_title_org(record):
    
    # This function removes project titles and organization names from abstracts
    
    """ ignores case to remove multi-word phrases in a particular order, especially those likely to run into other words,
    e.g. Institution university of washingtonPI mary williams. This doesn't work when titles or insititutions have escape characters in them, which is a bummer
    see for example ENHANCING THE USE OF NASA EARTH SCIENCE RESULTS / DATA / AND TECHNOLOGY BY ENGAGING THE FEDERATION OF EARTH SCIENCE INFORMATION PARTNERS COMMUNITIES OF
    PRACTICE IN TARGET AREAS OF INTEREST TO NASA THE FEDERATION OF EARTH SCIENCE INFORMATION PARTNERS (''FED"""
    
    title=record['PROJECT_TITLE']
    
    try:
        new_abstract=re.sub(title,'',record[wa],flags=re.IGNORECASE)      
        return re.sub(record['ORGANIZATION_NAME'],'',new_abstract,flags=re.IGNORECASE)   
    except:
        try:
            return re.sub(record['ORGANIZATION_NAME'],'',record[wa],flags=re.IGNORECASE)   
        except:
            return record[wa]
        
        
df[wa]=df.apply(lambda x: remove_title_org(x),axis=1)

In [ ]:
def remove_custom_words(record,col_to_clean):
    """Designates stopwords for a particular abstract that contain embedded info e.g. author names and removes them from a lowercase abstract"""
    fields_to_replace=[]
    if type(record[col_to_clean])!=list:
        return np.nan
    else:
        #Main PI
        #Adds all words in the pis names, excluding initials (hence why the commas and periods must be replaced)
        if pd.notnull(record['CONTACT_PI_PROJECT_LEADER']):
            fields_to_replace.extend([x.lower() for x in record['CONTACT_PI_PROJECT_LEADER'].replace(',','').replace('.','').replace('-',' ').split() if len(x)>1])
        #Additional PIs
        #For each pi, which are split by semicolons, and format is last,first;  #Sometimes a middle initial
        if pd.notnull(record['OTHER_PIS']):
            for i in record['OTHER_PIS'].split(';'):
                i=i.strip() #Remove whitespace
                i=i.replace('.','')#Periods for initials
                i=i.replace(',','')#Commas between last, first
                i=i.replace('-',' ')#Remove hyphen in hypenated names to make separate words once tokens.
                fields_to_replace.extend([x.lower().strip() for x in i.split() if len(x)>1])
        return [x.lower() for x in record[col_to_clean] if not x.lower() in fields_to_replace]

In [ ]:
# strip non-alphanum characters from the beginning and end of each abstract

df["working_abstract"] = [TextCleaning.strip_nonalnum(abstract) for abstract in df["working_abstract"]]
df = TextCleaning.drop_empties(df)

df['Start Char']=df[wa].apply(lambda x:x[0])
df['LAST_CHAR']=df[wa].apply(lambda x:x[-1])
df['nchar']=df[wa].apply(lambda x: len(x))

In [25]:
df.to_pickle("./clean_data_7-7-20.pkl")


In [8]:
# NOT RUNNING THIS

#####################
#Additional expressions we could choose to remove
#Identify abstracts with excessive amounts of other fields to uncover additional bad abstract types
#If we wanted to be on the safe side, some EDA makes me think we could remove anything with more than 
# 3 or 4 of these fields. It's where they start getting wonky.
###################

fields=['Principal Investigator','Program Director','Attachment','Instructions','Lines',
        'Space Provided','Performance Site','Organization','Key Personnel']
all_fields=fields.copy()
all_fields.extend([x.lower() for x in fields])
all_fields.extend([x.upper() for x in fields])
all_fields.extend(['PI','Form','Page','Title','.pdf','.doc'])

def count_up_fields(abstract):
    count=0
    for field in all_fields:
        if field in abstract:
            count+=1
    return count

df['Field Count']=df[wa].apply(count_up_fields)


In [176]:
# NOT RUNNING THIS

###########################
#Additional expressions we could remove, but there is a small possibility of some information being lost
##########################

#Issues: 'Close FormNextPrint PageAbout OMB Number']#This is usually ended with "Project summary", 
#so anything between those 2 can be delete, and #ended with a clause starting with 'Close FormProject' and ending in'Narrative File'

#expression1=re.compile('Close FormNext.*?Project Summary')
#expression2=re.compile('Close FormProject.*Narrative File')

def fix_abstract(abstract):
    if abstract.startswith('Close FormNext'):
        abstract=re.sub(expression1,'',abstract)
        return re.sub(expression2,'',abstract)
    else:
        return abstract

#df[wa]=df[wa].apply(fix_abstract)

#If ends in 'Description,', then go to last instance of PERFORMANCE (for Performance SITES), otherwise "KEY PERSONNEL", upper case, and cut all that follows

#expression1=re.compile('PERFORMANCE.*Description,$')
#expression2=re.compile('KEY PERSONNEL.*Description,$')

def apply_expressions(abstract):
    if abstract.endswith('Description,'):
        if re.search(expression1,abstract) != None:
            return re.sub(expression1,'',abstract)
        else:
            return re.sub(expression2,'',abstract)
    else:
        return abstract
    
#df[wa]=df[wa].apply(apply_expressions)

#expression1=re.compile('PERFORMANCE.*Page 3$')
#expression2=re.compile('KEY PERSONNEL.*Page 3,$')

def apply_expressions(abstract):
    if abstract.endswith('Description,'):
        if re.search(expression1,abstract) != None:
            return re.sub(expression1,'',abstract)
        else:
            return re.sub(expression2,'',abstract)
    else:
        return abstract
    
#df[wa]=df[wa].apply(apply_expressions)

